### Instructions:
##### Part 1:
Write two data descriptors:
- IntegerField -> only allows `integral` numbers between `minimum` and `maximum` value.
- CharField -> only allows `strings` with a `minimum` and `maximum` length.

Assume we're not using slots.

##### Part 2:

Refactor your code and createa a `BaseValidator` class that will handle the common functionality.

Then change your `IntegerField` and `CharField` descriptors to inherit from BaseValidator.


If you haven't coded your descriptors that way already, make sure you can also omit one or both of the `miminum` and `maximum` values where it makes sense.

For example we may want to specify a string field that has no `maximum` limit, or we may want an integer field that has an upper bound, but no lower bound, or maybe no bounds at all.